# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [7]:
class Matrix:
    def __init__(self, data, m=None):
        if isinstance(data, int) and isinstance(m, int):
            self.n = data
            self.m = m
            self.matrix = [[0 for _ in range(m)] for _ in range(data)]
        elif isinstance(data, list):
            if all(isinstance(row, list) for row in data) and len(set(len(row) for row in data)) == 1:
                self.n = len(data)
                self.m = len(data[0])
                self.matrix = [row[:] for row in data]
            else:
                raise ValueError("All rows must have the same number of columns")
        else:
            raise TypeError("Invalid initialization parameters")

    def __getitem__(self, index):
        if isinstance(index, tuple) and len(index) == 2:
            i, j = index
            return self.matrix[i][j]
        return self.matrix[index]

    def __setitem__(self, index, value):
        if isinstance(index, tuple) and len(index) == 2:
            i, j = index
            self.matrix[i][j] = value
        else:
            self.matrix[index] = value
            
    def assign(self, other):
        if isinstance(other, Matrix):
            if self.n == other.n and self.m == other.m:
                self.matrix = []
                for row in other.matrix:
                    new_row = []
                    for elem in row:
                        new_row.append(elem)
                    self.matrix.append(new_row)
            else:
                raise ValueError("Matrix dimensions must match for assignment")
        elif isinstance(other, list):
            if len(other) == self.n and all(len(row) == self.m for row in other):
                self.matrix = []
                for row in other:
                    new_row = []
                    for elem in row:
                        new_row.append(elem)
                    self.matrix.append(new_row)
            else:
                raise ValueError("List dimensions must match matrix dimensions")
        else:
            raise TypeError("Assignment must be from a Matrix or a list of lists")

    def __repr__(self):
        return "\n".join([" ".join(map(str, row)) for row in self.matrix])

In [8]:
# Test 1: Initialize with size (n, m)
M1 = Matrix(3, 4)  # 3x4 zero matrix
print("Test 1: Initialize with size (3,4)")
print(M1)

# Test 2: Initialize with a list of lists
M2 = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print("\nTest 2: Initialize with a list of lists")
print(M2)

# Test 3: Attempt to initialize with irregular row sizes (should raise ValueError)
try:
    M3 = Matrix([[1, 2], [3, 4, 5]])
except ValueError as e:
    print("\nTest 3: Irregular row sizes")
    print("Passed -", e)

# Test 4: Accessing elements using M[i][j] and M[i, j]
print("\nTest 4: Accessing elements")
print("M2[1][2]:", M2[1][2])
print("M2[1, 2]:", M2[1, 2])

# Test 5: Setting elements using M[i][j] and M[i, j]
M2[1][2] = 99
M2[1, 1] = 88
print("\nTest 5: Setting elements")
print(M2)

# Test 6: Assigning one matrix to another
M4 = Matrix(3, 3)
M4.assign(M2)
print("\nTest 6: Assigning one matrix to another")
print(M4)

# Test 7: Assigning with a list of lists of correct dimensions
M5 = Matrix(3, 3)
M5.assign([[10, 20, 30], [40, 50, 60], [70, 80, 90]])
print("\nTest 7: Assigning with a list of lists")
print(M5)

# Test 8: Assigning with a mismatched size (should raise ValueError)
try:
    M5.assign([[1, 2], [3, 4]])
except ValueError as e:
    print("\nTest 8: Assigning with mismatched size")
    print("Passed -", e)


Test 1: Initialize with size (3,4)
0 0 0 0
0 0 0 0
0 0 0 0

Test 2: Initialize with a list of lists
1 2 3
4 5 6
7 8 9

Test 3: Irregular row sizes
Passed - All rows must have the same number of columns

Test 4: Accessing elements
M2[1][2]: 6
M2[1, 2]: 6

Test 5: Setting elements
1 2 3
4 88 99
7 8 9

Test 6: Assigning one matrix to another
1 2 3
4 88 99
7 8 9

Test 7: Assigning with a list of lists
10 20 30
40 50 60
70 80 90

Test 8: Assigning with mismatched size
Passed - List dimensions must match matrix dimensions


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [9]:
class Matrix:
    def __init__(self, data, m=None):
        if isinstance(data, int) and isinstance(m, int):
            self.n = data
            self.m = m
            self.matrix = [[0 for _ in range(m)] for _ in range(data)]
        elif isinstance(data, list):
            if all(isinstance(row, list) for row in data) and len(set(len(row) for row in data)) == 1:
                self.n = len(data)
                self.m = len(data[0])
                self.matrix = [row[:] for row in data]
            else:
                raise ValueError("All rows must have the same number of columns")
        else:
            raise TypeError("Invalid initialization parameters")

    def __getitem__(self, index):
        if isinstance(index, tuple):
            i, j = index
            if isinstance(i, slice) and isinstance(j, slice):
                return [row[j] for row in self.matrix[i]]
            elif isinstance(i, slice):
                return [row[j] for row in self.matrix[i]]
            elif isinstance(j, slice):
                return self.matrix[i][j]
            else:
                return self.matrix[i][j]
        return self.matrix[index]

    def __setitem__(self, index, value):
        if isinstance(index, tuple) and len(index) == 2:
            i, j = index
            self.matrix[i][j] = value
        else:
            self.matrix[index] = value
            
    def assign(self, other):
        if isinstance(other, Matrix):
            if self.n == other.n and self.m == other.m:
                self.matrix = []
                for row in other.matrix:
                    new_row = []
                    for elem in row:
                        new_row.append(elem)
                    self.matrix.append(new_row)
            else:
                raise ValueError("Matrix dimensions must match for assignment")
        elif isinstance(other, list):
            if len(other) == self.n and all(len(row) == self.m for row in other):
                self.matrix = []
                for row in other:
                    new_row = []
                    for elem in row:
                        new_row.append(elem)
                    self.matrix.append(new_row)
            else:
                raise ValueError("List dimensions must match matrix dimensions")
        else:
            raise TypeError("Assignment must be from a Matrix or a list of lists")

    def __repr__(self):
        return "\n".join([" ".join(map(str, row)) for row in self.matrix])

    def shape(self):
        return self.n, self.m

    def transpose(self):
        transposed_matrix = []
        for i in range(self.m):
            row = []
            for j in range(self.n):
                row.append(self.matrix[j][i])
            transposed_matrix.append(row)
        return Matrix(transposed_matrix)

    def row(self, n):
        row_matrix = []
        for elem in self.matrix[n]:
            row_matrix.append(elem)
        return Matrix([row_matrix])

    def column(self, n):
        column_matrix = []
        for i in range(self.n):
            row = []
            row.append(self.matrix[i][n])
            column_matrix.append(row)
        return Matrix(column_matrix)

    def to_list(self):
        list_matrix = []
        for row in self.matrix:
            new_row = []
            for elem in row:
                new_row.append(elem)
            list_matrix.append(new_row)
        return list_matrix

    def block(self, n_0, n_1, m_0, m_1):
        block_matrix = []
        for i in range(n_0, n_1):
            row = []
            for j in range(m_0, m_1):
                row.append(self.matrix[i][j])
            block_matrix.append(row)
        return Matrix(block_matrix)

In [10]:
# Test cases for the Matrix class

# 1. Testing shape() method
print("Testing shape() method:")
m1 = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print("Matrix 1 shape:", m1.shape())  # Expected output: (3, 3)

m2 = Matrix([[1, 2, 3, 4], [5, 6, 7, 8]])
print("Matrix 2 shape:", m2.shape())  # Expected output: (2, 4)

m3 = Matrix([[1, 2, 3, 4, 5]])
print("Matrix 3 shape:", m3.shape())  # Expected output: (1, 5)

print("\n" + "-" * 40 + "\n")

# 2. Testing transpose() method
print("Testing transpose() method:")
print("Original Matrix 1:")
print(m1.to_list())
print("Transposed Matrix 1:")
print(m1.transpose())  # Expected output: [[1, 4, 7], [2, 5, 8], [3, 6, 9]]

print("Original Matrix 2:")
print(m2.to_list())
print("Transposed Matrix 2:")
print(m2.transpose())  # Expected output: [[1, 5], [2, 6], [3, 7], [4, 8]]

print("Original Matrix 3:")
print(m3.to_list())
print("Transposed Matrix 3:")
print(m3.transpose())  # Expected output: [[1], [2], [3], [4], [5]]

print("\n" + "-" * 40 + "\n")

# 3. Testing row() method
print("Testing row() method:")
print("Matrix 1, Row 0:", m1.row(0))  # Expected output: [1, 2, 3]
print("Matrix 2, Row 1:", m2.row(1))  # Expected output: [5, 6, 7, 8]
print("Matrix 3, Row 0:", m3.row(0))  # Expected output: [1, 2, 3, 4, 5]

print("\n" + "-" * 40 + "\n")

# 4. Testing column() method
print("Testing column() method:")
print("Matrix 1, Column 0:")
print(m1.column(0))  # Expected output: [1, 4, 7]
print("Matrix 2, Column 1:")
print(m2.column(1))  # Expected output: [2, 6]
print("Matrix 3, Column 2:")
print(m3.column(2))  # Expected output: [3]

print("\n" + "-" * 40 + "\n")

# 5. Testing to_list() method
print("Testing to_list() method:")
print("Matrix 1 as list:")
print(m1.to_list())  # Expected output: [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
print("Matrix 2 as list:")
print(m2.to_list())  # Expected output: [[1, 2, 3, 4], [5, 6, 7, 8]]
print("Matrix 3 as list:")
print(m3.to_list())  # Expected output: [[1, 2, 3, 4, 5]]

print("\n" + "-" * 40 + "\n")

# 6. Testing block() method
print("Testing block() method:")
print("Matrix 1 block (0,2) x (0,2):")
print(m1.block(0, 2, 0, 2))  # Expected output: [[1, 2], [4, 5]]
print("Matrix 2 block (0,1) x (2,4):")
print(m2.block(0, 1, 2, 4))  # Expected output: [[3, 4]]
print("Matrix 1 block (0,2) x (1,2):")
print(m1.block(0, 2, 1, 2))  # Expected output: [[2], [5]]
# Test cases for the Matrix class

# 1. Testing shape() method
print("Testing shape() method:")
m1 = Matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print("Matrix 1 shape:", m1.shape())  # Expected output: (3, 3)

m2 = Matrix([[1, 2, 3, 4], [5, 6, 7, 8]])
print("Matrix 2 shape:", m2.shape())  # Expected output: (2, 4)

m3 = Matrix([[1, 2, 3, 4, 5]])
print("Matrix 3 shape:", m3.shape())  # Expected output: (1, 5)

print("\n" + "-" * 40 + "\n")

# 2. Testing transpose() method
print("Testing transpose() method:")
print("Original Matrix 1:")
print(m1.to_list())
print("Transposed Matrix 1:")
print(m1.transpose())  # Expected output: [[1, 4, 7], [2, 5, 8], [3, 6, 9]]

print("Original Matrix 2:")
print(m2.to_list())
print("Transposed Matrix 2:")
print(m2.transpose())  # Expected output: [[1, 5], [2, 6], [3, 7], [4, 8]]

print("Original Matrix 3:")
print(m3.to_list())
print("Transposed Matrix 3:")
print(m3.transpose())  # Expected output: [[1], [2], [3], [4], [5]]

print("\n" + "-" * 40 + "\n")

# 3. Testing row() method
print("Testing row() method:")
print("Matrix 1, Row 0:", m1.row(0))  # Expected output: [1, 2, 3]
print("Matrix 2, Row 1:", m2.row(1))  # Expected output: [5, 6, 7, 8]
print("Matrix 3, Row 0:", m3.row(0))  # Expected output: [1, 2, 3, 4, 5]

print("\n" + "-" * 40 + "\n")

# 4. Testing column() method
print("Testing column() method:")
print("Matrix 1, Column 0:")
print(m1.column(0))  # Expected output: [1, 4, 7]
print("Matrix 2, Column 1:")
print(m2.column(1))  # Expected output: [2, 6]
print("Matrix 3, Column 2:")
print(m3.column(2))  # Expected output: [3]

print("\n" + "-" * 40 + "\n")

# 5. Testing to_list() method
print("Testing to_list() method:")
print("Matrix 1 as list:")
print(m1.to_list())  # Expected output: [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
print("Matrix 2 as list:")
print(m2.to_list())  # Expected output: [[1, 2, 3, 4], [5, 6, 7, 8]]
print("Matrix 3 as list:")
print(m3.to_list())  # Expected output: [[1, 2, 3, 4, 5]]

print("\n" + "-" * 40 + "\n")

# 6. Testing block() method
print("Testing block() method:")
print("Matrix 1 block (0,2) x (0,2):")
print(m1.block(0, 2, 0, 2))  # Expected output: [[1, 2], [4, 5]]
print("Matrix 2 block (0,1) x (2,4):")
print(m2.block(0, 1, 2, 4))  # Expected output: [[3, 4]]
print("Matrix 1 block (0,2) x (1,2):")
print(m1.block(0, 2, 1, 2))  # Expected output: [[2], [5]]

print("\n" + "-" * 40 + "\n")

#Checking get item for slicing:
print("Testing slicing for __getitem__ method:")
print(m1[1, 2])       # Output: 6 (single element)
print(m1[0, :])       # Output: [1, 2, 3] (full first row)
print(m1[:, 1])       # Output: [2, 5, 8] (full second column)
print(m1[0:2, 1:])    # Output: [[2, 3], [5, 6]] (submatrix)

Testing shape() method:
Matrix 1 shape: (3, 3)
Matrix 2 shape: (2, 4)
Matrix 3 shape: (1, 5)

----------------------------------------

Testing transpose() method:
Original Matrix 1:
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
Transposed Matrix 1:
1 4 7
2 5 8
3 6 9
Original Matrix 2:
[[1, 2, 3, 4], [5, 6, 7, 8]]
Transposed Matrix 2:
1 5
2 6
3 7
4 8
Original Matrix 3:
[[1, 2, 3, 4, 5]]
Transposed Matrix 3:
1
2
3
4
5

----------------------------------------

Testing row() method:
Matrix 1, Row 0: 1 2 3
Matrix 2, Row 1: 5 6 7 8
Matrix 3, Row 0: 1 2 3 4 5

----------------------------------------

Testing column() method:
Matrix 1, Column 0:
1
4
7
Matrix 2, Column 1:
2
6
Matrix 3, Column 2:
3

----------------------------------------

Testing to_list() method:
Matrix 1 as list:
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]
Matrix 2 as list:
[[1, 2, 3, 4], [5, 6, 7, 8]]
Matrix 3 as list:
[[1, 2, 3, 4, 5]]

----------------------------------------

Testing block() method:
Matrix 1 block (0,2) x (0,2):
1 2
4 5
Ma

3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [11]:
def constant(n, m, c):
    matrix = Matrix(n,m)
    for i in range(n):
        for j in range(m):
            matrix[i][j] = c
    return matrix

def zeros(n, m):
    matrix = Matrix(n, m)
    return matrix

def ones(n, m):
    matrix = Matrix(n,m)
    for i in range(n):
        for j in range(m):
            matrix[i][j] = 1
    return matrix

def eye(n):
    matrix = Matrix(n, n)
    for i in range(n):
        for j in range(n):
            if i == j:
                matrix[i][j] = 1
    return matrix

In [12]:
#Test of Solution
print("Constant Matrix (3x4, value 5):")
print(constant(3, 4, 5.1))

print("\nZero Matrix (2x3):")
print(zeros(2, 3))

print("\nOnes Matrix (3x3):")
print(ones(3, 3))

print("\nIdentity Matrix (4x4):")
print(eye(4))


Constant Matrix (3x4, value 5):
5.1 5.1 5.1 5.1
5.1 5.1 5.1 5.1
5.1 5.1 5.1 5.1

Zero Matrix (2x3):
0 0 0
0 0 0

Ones Matrix (3x3):
1 1 1
1 1 1
1 1 1

Identity Matrix (4x4):
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [13]:
class Matrix:
    def __init__(self, data, m=None):
        if isinstance(data, int) and isinstance(m, int):
            self.n = data
            self.m = m
            self.matrix = [[0 for _ in range(m)] for _ in range(data)]
        elif isinstance(data, list):
            if all(isinstance(row, list) for row in data) and len(set(len(row) for row in data)) == 1:
                self.n = len(data)
                self.m = len(data[0])
                self.matrix = [row[:] for row in data]
            else:
                raise ValueError("All rows must have the same number of columns")
        else:
            raise TypeError("Invalid initialization parameters")

    def __getitem__(self, index):
        if isinstance(index, tuple):
            i, j = index
            if isinstance(i, slice) and isinstance(j, slice):
                return [row[j] for row in self.matrix[i]]
            elif isinstance(i, slice):
                return [row[j] for row in self.matrix[i]]
            elif isinstance(j, slice):
                return self.matrix[i][j]
            else:
                return self.matrix[i][j]
        return self.matrix[index]

    def __setitem__(self, index, value):
        if isinstance(index, tuple) and len(index) == 2:
            i, j = index
            self.matrix[i][j] = value
        else:
            self.matrix[index] = value
            
    def assign(self, other):
        if isinstance(other, Matrix):
            if self.n == other.n and self.m == other.m:
                self.matrix = []
                for row in other.matrix:
                    new_row = []
                    for elem in row:
                        new_row.append(elem)
                    self.matrix.append(new_row)
            else:
                raise ValueError("Matrix dimensions must match for assignment")
        elif isinstance(other, list):
            if len(other) == self.n and all(len(row) == self.m for row in other):
                self.matrix = []
                for row in other:
                    new_row = []
                    for elem in row:
                        new_row.append(elem)
                    self.matrix.append(new_row)
            else:
                raise ValueError("List dimensions must match matrix dimensions")
        else:
            raise TypeError("Assignment must be from a Matrix or a list of lists")

    def __repr__(self):
        return "\n".join([" ".join(map(str, row)) for row in self.matrix])

    def shape(self):
        return self.n, self.m

    def transpose(self):
        transposed_matrix = []
        for i in range(self.m):
            row = []
            for j in range(self.n):
                row.append(self.matrix[j][i])
            transposed_matrix.append(row)
        return Matrix(transposed_matrix)

    def row(self, n):
        row_matrix = []
        for elem in self.matrix[n]:
            row_matrix.append(elem)
        return Matrix([row_matrix])

    def column(self, n):
        column_matrix = []
        for i in range(self.n):
            row = []
            row.append(self.matrix[i][n])
            column_matrix.append(row)
        return Matrix(column_matrix)

    def to_list(self):
        list_matrix = []
        for row in self.matrix:
            new_row = []
            for elem in row:
                new_row.append(elem)
            list_matrix.append(new_row)
        return list_matrix

    def block(self, n_0, n_1, m_0, m_1):
        block_matrix = []
        for i in range(n_0, n_1):
            row = []
            for j in range(m_0, m_1):
                row.append(self.matrix[i][j])
            block_matrix.append(row)
        return Matrix(block_matrix)

    def scalarmul(self, c):
        result = []
        for i in range(self.n):
            row = []
            for j in range(self.m):
                row.append(self.matrix[i][j] * c)
            result.append(row)
        return Matrix(result)

    def add(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions for addition")
        result = []
        for i in range(self.n):
            row = []
            for j in range(self.m):
                row.append(self.matrix[i][j] + N.matrix[i][j])
            result.append(row)
        return Matrix(result)

    def sub(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions for subtraction")
        result = []
        for i in range(self.n):
            row = []
            for j in range(self.m):
                row.append(self.matrix[i][j] - N.matrix[i][j])
            result.append(row)
        return Matrix(result)

    def mat_mult(self, N):
        if self.m != N.n:
            raise ValueError("Number of columns in M must match number of rows in N for matrix multiplication")
        result = []
        for i in range(self.n):
            row = []
            for j in range(N.m):
                sum_product = 0
                for k in range(self.m):
                    sum_product += self.matrix[i][k] * N.matrix[k][j]
                row.append(sum_product)
            result.append(row)
        return Matrix(result)

    def element_mult(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication")
        result = []
        for i in range(self.n):
            row = []
            for j in range(self.m):
                row.append(self.matrix[i][j] * N.matrix[i][j])
            result.append(row)
        return Matrix(result)

    def equals(self, N):
        if self.shape() != N.shape():
            return False
        for i in range(self.n):
            for j in range(self.m):
                if self.matrix[i][j] != N.matrix[i][j]:
                    return False
        return True

In [14]:
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[1, 2, 3], [4, 5, 6]])  
D = Matrix([[1], [2], [3]])  
scalar = 2

print("Matrix A:")
print(A)

print("\nMatrix B:")
print(B)

print("\nMatrix C (Different Dimension):")
print(C)

print("\nMatrix D (Different Dimension - Column Vector):")
print(D)

print("\nScalar Multiplication (A * 2):")
print(A.scalarmul(scalar))


print("\nAddition (A + B):")
print(A.add(B))

print("\nSubtraction (A - B):")
print(A.sub(B))

try:
    print("\nAddition (A + C) - Should Fail:")
    print(A.add(C))
except Exception as e:
    print(f"Error: {e}")

print("\nMatrix Multiplication (A * B):")
print(A.mat_mult(B))

print("\nMatrix Multiplication (A * C):")
print(A.mat_mult(C))

print("\nMatrix Multiplication (C * D):")
print(C.mat_mult(D))

print("\nElement-wise Multiplication (A .* B):")
print(A.element_mult(B))

try:
    print("\nElement-wise Multiplication (A .* C) - Should Fail:")
    print(A.element_mult(C))
except Exception as e:
    print(f"Error: {e}")

print("\nEquality Check (A == B):", A.equals(B))
print("\nEquality Check (A == C):", A.equals(C))
print("\nEquality Check (A == A):", A.equals(A))


Matrix A:
1 2
3 4

Matrix B:
5 6
7 8

Matrix C (Different Dimension):
1 2 3
4 5 6

Matrix D (Different Dimension - Column Vector):
1
2
3

Scalar Multiplication (A * 2):
2 4
6 8

Addition (A + B):
6 8
10 12

Subtraction (A - B):
-4 -4
-4 -4

Addition (A + C) - Should Fail:
Error: Matrices must have the same dimensions for addition

Matrix Multiplication (A * B):
19 22
43 50

Matrix Multiplication (A * C):
9 12 15
19 26 33

Matrix Multiplication (C * D):
14
32

Element-wise Multiplication (A .* B):
5 12
21 32

Element-wise Multiplication (A .* C) - Should Fail:
Error: Matrices must have the same dimensions for element-wise multiplication

Equality Check (A == B): False

Equality Check (A == C): False

Equality Check (A == A): True


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [15]:
class Matrix:
    def __init__(self, data, m=None):
        if isinstance(data, int) and isinstance(m, int):
            self.n = data
            self.m = m
            self.matrix = [[0 for _ in range(m)] for _ in range(data)]
        elif isinstance(data, list):
            if all(isinstance(row, list) for row in data) and len(set(len(row) for row in data)) == 1:
                self.n = len(data)
                self.m = len(data[0])
                self.matrix = [row[:] for row in data]
            else:
                raise ValueError("All rows must have the same number of columns")
        else:
            raise TypeError("Invalid initialization parameters")

    def __getitem__(self, index):
        if isinstance(index, tuple):
            i, j = index
            if isinstance(i, slice) and isinstance(j, slice):
                return [row[j] for row in self.matrix[i]]
            elif isinstance(i, slice):
                return [row[j] for row in self.matrix[i]]
            elif isinstance(j, slice):
                return self.matrix[i][j]
            else:
                return self.matrix[i][j]
        return self.matrix[index]

    def __setitem__(self, index, value):
        if isinstance(index, tuple) and len(index) == 2:
            i, j = index
            self.matrix[i][j] = value
        else:
            self.matrix[index] = value
            
    def assign(self, other):
        if isinstance(other, Matrix):
            if self.n == other.n and self.m == other.m:
                self.matrix = []
                for row in other.matrix:
                    new_row = []
                    for elem in row:
                        new_row.append(elem)
                    self.matrix.append(new_row)
            else:
                raise ValueError("Matrix dimensions must match for assignment")
        elif isinstance(other, list):
            if len(other) == self.n and all(len(row) == self.m for row in other):
                self.matrix = []
                for row in other:
                    new_row = []
                    for elem in row:
                        new_row.append(elem)
                    self.matrix.append(new_row)
            else:
                raise ValueError("List dimensions must match matrix dimensions")
        else:
            raise TypeError("Assignment must be from a Matrix or a list of lists")

    def __repr__(self):
        return "\n".join([" ".join(map(str, row)) for row in self.matrix])

    def shape(self):
        return self.n, self.m

    def transpose(self):
        transposed_matrix = []
        for i in range(self.m):
            row = []
            for j in range(self.n):
                row.append(self.matrix[j][i])
            transposed_matrix.append(row)
        return Matrix(transposed_matrix)

    def row(self, n):
        row_matrix = []
        for elem in self.matrix[n]:
            row_matrix.append(elem)
        return Matrix([row_matrix])

    def column(self, n):
        column_matrix = []
        for i in range(self.n):
            row = []
            row.append(self.matrix[i][n])
            column_matrix.append(row)
        return Matrix(column_matrix)

    def to_list(self):
        list_matrix = []
        for row in self.matrix:
            new_row = []
            for elem in row:
                new_row.append(elem)
            list_matrix.append(new_row)
        return list_matrix

    def block(self, n_0, n_1, m_0, m_1):
        block_matrix = []
        for i in range(n_0, n_1):
            row = []
            for j in range(m_0, m_1):
                row.append(self.matrix[i][j])
            block_matrix.append(row)
        return Matrix(block_matrix)

    def scalarmul(self, c):
        result = []
        for i in range(self.n):
            row = []
            for j in range(self.m):
                row.append(self.matrix[i][j] * c)
            result.append(row)
        return Matrix(result)

    def add(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions for addition")
        result = []
        for i in range(self.n):
            row = []
            for j in range(self.m):
                row.append(self.matrix[i][j] + N.matrix[i][j])
            result.append(row)
        return Matrix(result)

    def sub(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions for subtraction")
        result = []
        for i in range(self.n):
            row = []
            for j in range(self.m):
                row.append(self.matrix[i][j] - N.matrix[i][j])
            result.append(row)
        return Matrix(result)

    def mat_mult(self, N):
        if self.m != N.n:
            raise ValueError("Number of columns in M must match number of rows in N for matrix multiplication")
        result = []
        for i in range(self.n):
            row = []
            for j in range(N.m):
                sum_product = 0
                for k in range(self.m):
                    sum_product += self.matrix[i][k] * N.matrix[k][j]
                row.append(sum_product)
            result.append(row)
        return Matrix(result)

    def element_mult(self, N):
        if self.shape() != N.shape():
            raise ValueError("Matrices must have the same dimensions for element-wise multiplication")
        result = []
        for i in range(self.n):
            row = []
            for j in range(self.m):
                row.append(self.matrix[i][j] * N.matrix[i][j])
            result.append(row)
        return Matrix(result)

    def equals(self, N):
        if self.shape() != N.shape():
            return False
        for i in range(self.n):
            for j in range(self.m):
                if self.matrix[i][j] != N.matrix[i][j]:
                    return False
        return True

    def __mul__(self, other):
        if isinstance(other, (int, float)):  # Scalar multiplication
            return self.scalarmul(other)
        elif isinstance(other, Matrix):  # Matrix multiplication
            return self.mat_mult(other)
        else:
            raise TypeError("Unsupported operand type for *: 'Matrix' and '{}'".format(type(other).__name__))

    def __rmul__(self, other):
        return self * other  # Allows scalar multiplication from both sides (2 * M and M * 2)

    def __add__(self, other):
        if isinstance(other, Matrix):
            return self.add(other)
        raise TypeError("Unsupported operand type for +: 'Matrix' and '{}'".format(type(other).__name__))

    def __sub__(self, other):
        if isinstance(other, Matrix):
            return self.sub(other)
        raise TypeError("Unsupported operand type for -: 'Matrix' and '{}'".format(type(other).__name__))

    def __eq__(self, other):
        if isinstance(other, Matrix):
            return self.equals(other)
        return False

    def __setattr__(self, name, value):
        if name == "matrix":
            if hasattr(self, "n") and hasattr(self, "m"):
                if len(value) != self.n or any(len(row) != self.m for row in value):
                    raise ValueError("Assignment must match matrix dimensions")
        super().__setattr__(name, value)


In [16]:
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])

print("Matrix A:")
print(A)

print("\nMatrix B:")
print(B)

# Scalar Multiplication
print("\nScalar Multiplication (2 * A):")
print(2 * A)

print("\nScalar Multiplication (A * 2):")
print(A * 2)

# Matrix Addition
print("\nMatrix Addition (A + B):")
print(A + B)

# Matrix Subtraction
print("\nMatrix Subtraction (A - B):")
print(A - B)

# Matrix Multiplication
print("\nMatrix Multiplication (A * B):")
print(A * B)

# Equality Check
print("\nCheck if A == B:", A == B)

# Assignment
print("\nAssign B to A")
A = B
print("New A:")
print(A)


Matrix A:
1 2
3 4

Matrix B:
5 6
7 8

Scalar Multiplication (2 * A):
2 4
6 8

Scalar Multiplication (A * 2):
2 4
6 8

Matrix Addition (A + B):
6 8
10 12

Matrix Subtraction (A - B):
-4 -4
-4 -4

Matrix Multiplication (A * B):
19 22
43 50

Check if A == B: False

Assign B to A
New A:
5 6
7 8


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [17]:
# Create example matrices
A = Matrix([[1, 2], [3, 4]])
B = Matrix([[5, 6], [7, 8]])
C = Matrix([[2, 0], [1, 3]])
I = eye(2)

# Associativity of Matrix Multiplication: (AB)C = A(BC)
left_side = (A * B) * C  # (AB)C
right_side = A * (B * C)  # A(BC)
print("Associativity: (AB)C == A(BC) ?", left_side == right_side)

# Distributive Property: A(B + C) = AB + AC
left_side = A * (B + C)  # A(B+C)
right_side = (A * B) + (A * C)  # AB + AC
print("Distributive Property: A(B + C) == AB + AC ?", left_side == right_side)

# Non-Commutativity: AB ≠ BA
AB = A * B
BA = B * A
print("Non-Commutativity: AB == BA ?", AB == BA)

# Identity Matrix: AI = A
AI = A * I
print("Identity Matrix Property: AI == A ?", AI == A)


Associativity: (AB)C == A(BC) ? True
Distributive Property: A(B + C) == AB + AC ? True
Non-Commutativity: AB == BA ? False
Identity Matrix Property: AI == A ? True
